TODO

[X] extract frames from video (.mp4)

[ ] create PyTorch Dataset

[ ] seaparate into training , validation and test dataset

In [1]:
import cv2

**extract frames from video and save them as .jpeg**

source: https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames

list of supported formats for extracted images: https://docs.opencv.org/3.4/d4/da8/group__imgcodecs.html#ga288b8b3da0892bd651fce07b3bbd3a56

In [10]:
vidcap = cv2.VideoCapture('/home/advo/dev/DMS_android/PyTorch/Resources/VID_20210530_214905012.mp4')
success,image = vidcap.read()
count = 0
while success and count < 20:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True


**Implement DataLoader**

Source: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [3]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [23]:
class DriverFaceDataset(Dataset):
    """Driver Face dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.labels_frame.iloc[idx, 0])
        image = io.imread(img_name)
        if self.labels_frame.iloc[idx,1] == 1:
            sample = {'image': image, 'driver_state': 'focused'}
        else:
            sample = {'image': image, 'driver_state': 'distracted'}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [24]:
driver_dataset = DriverFaceDataset(csv_file="/home/advo/dev/DMS_android/PyTorch/Resources/labels.csv",
                                  root_dir="/home/advo/dev/DMS_android/PyTorch/Resources/images")

In [25]:
driver_dataset[2]['driver_state']

'distracted'

In [26]:
dataloader = DataLoader(driver_dataset, batch_size=4,
                        shuffle=True, num_workers=0)